# Regression Model

In [76]:
# Imports
import pandas as pd
from data_cleaning import clean_car_data
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler

In [83]:
# Import the Data
cars_raw = pd.read_csv("USA_cars_datasets.csv")

In [84]:
# Clean the data
cars = clean_car_data(cars_raw)

In [85]:
# One-hot Encoding for categorical variables

# Determine categorical columns
category_columns = cars.select_dtypes('category').columns

# Drop categorical columns
cars = pd.get_dummies(cars, columns=category_columns, drop_first=True)


In [87]:
# Separate the features and target variable
x = cars.drop(columns='price')
y = cars['price']

In [88]:
# Standardize the numeric variables
scaler = MinMaxScaler()

# Identify numeric columns
number_columns = x.select_dtypes('number').columns

# Standardize numeric features
x = scaler.fit_transform(cars[number_columns])

# Standardize the target variable
y = scaler.fit_transform(y['price'])

KeyError: 'price'

In [43]:
# Partition the Data
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=42)

In [ ]:
# Implement Cross-Validation

In [44]:
# Make a Regression Model
model = LinearRegression()
#model.fit(x_train, x_test)

ValueError: could not convert string to float: 'nissan'